<table class="tfo-notebook-buttons" align="center">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/PracticalDL/Practical-Deep-Learning-Book/blob/master/code/chapter-5/4-keras-tuner.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/PracticalDL/Practical-Deep-Learning-Book/blob/master/code/chapter-5/4-keras-tuner.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>

In [1]:
try:
    # Mount Google Drive
    from google.colab import drive

    drive.mount("/content/gdrive")

    IS_COLAB_ENV = True
except:
    IS_COLAB_ENV = False
IS_COLAB_ENV

Mounted at /content/gdrive


True

# Keras Tuner

With so many potential combinations of hyperparameters to tune, coming up with the best model can be a tedious process. Often two or more parameters might have correlated effects on the overall speed of convergence as well as validation accuracy, so tuning one at a time might not lead to the best model.  And if curiosity gets the best of us, we might want to experimentation on all the hyperparameters together!  

Keras Tuner comes to automate this hyperparameter search. We define a search algorithm, the potential values that each parameter can take (e.g. discrete values or a range), our target object to maximize (e.g. validation accuracy) and sit back to see the program start training. Keras Tuner will conduct multiple experiments changing the parameters on our behalf, storing the metadata of the best model. The following code example adapted from Keras Tuner documentation showcases searching through the different model architectures (varying in the number of layers between 2 and 10) as well as varying the learning rate (between 0.1 and 0.001).

In [2]:
!pip install keras-tuner

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.1/176.1 kB 3.3 MB/s eta 0:00:00


In [3]:
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np

In [4]:
from kerastuner.engine.hypermodel import HyperModel
from kerastuner.engine.hyperparameters import HyperParameters
from kerastuner.tuners import RandomSearch

<ipython-input-4-52e813ccbcd6>:1: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner.engine.hypermodel import HyperModel


In [5]:
# Input data
(x, y), (val_x, val_y) = keras.datasets.mnist.load_data()
x = x.astype("float32") / 255.0
val_x = val_x.astype("float32") / 255.0

11490434/11490434 [==============================] - 0s 0us/step


In [6]:
# Defining hyper parameters
hp = HyperParameters()
hp.Choice("learning_rate", [0.1, 0.001])
hp.Int("num_layers", 2, 10)

2

In [7]:
# Defining model with expandable number of layers
def build_model(hp):
    model = keras.Sequential()
    model.add(layers.Flatten(input_shape=(28, 28)))
    for i in range(hp.get("num_layers")):
        model.add(layers.Dense(32, activation="relu"))
    model.add(layers.Dense(10, activation="softmax"))
    model.compile(
        optimizer=keras.optimizers.Adam(hp.get("learning_rate")),
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"],
    )
    return model

In [8]:
hypermodel = RandomSearch(
    build_model,
    max_trials=20,  # Number of combinations allowed
    hyperparameters=hp,
    allow_new_entries=False,
    objective="val_accuracy",
)

In [9]:
hypermodel.search(x=x, y=y, epochs=5, validation_data=(val_x, val_y))

Trial 18 Complete [00h 00m 37s]
val_accuracy: 0.11349999904632568

Best val_accuracy So Far: 0.9672999978065491
Total elapsed time: 00h 11m 18s


In [10]:
# Show summary of overall best model
hypermodel.results_summary()

Results summary
Results in ./untitled_project
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 15 summary
Hyperparameters:
learning_rate: 0.001
num_layers: 3
Score: 0.9672999978065491

Trial 14 summary
Hyperparameters:
learning_rate: 0.001
num_layers: 4
Score: 0.9663000106811523

Trial 02 summary
Hyperparameters:
learning_rate: 0.001
num_layers: 2
Score: 0.964900016784668

Trial 09 summary
Hyperparameters:
learning_rate: 0.001
num_layers: 8
Score: 0.9639999866485596

Trial 11 summary
Hyperparameters:
learning_rate: 0.001
num_layers: 5
Score: 0.9639000296592712

Trial 07 summary
Hyperparameters:
learning_rate: 0.001
num_layers: 10
Score: 0.9635000228881836

Trial 13 summary
Hyperparameters:
learning_rate: 0.001
num_layers: 6
Score: 0.9624999761581421

Trial 16 summary
Hyperparameters:
learning_rate: 0.001
num_layers: 7
Score: 0.9623000025749207

Trial 06 summary
Hyperparameters:
learning_rate: 0.001
num_layers: 9
Score: 0.9560999870300293

Trial 01 summary
H